In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [2]:
import getpass
if 'ec2' in getpass.getuser():
    print('On AWS')
    path_datasets = os.getcwd() + '/input/'
else:
    print('On Kaggle')
    path_datasets = '/kaggle/input/learn-together/'

On AWS


In [3]:
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score
import seaborn as sns
%matplotlib inline

import h2o
from h2o.automl import H2OAutoML

h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321 . connected.


H2O cluster uptime:,31 mins 11 secs
H2O cluster timezone:,Etc/UTC
H2O data parsing timezone:,UTC
H2O cluster version:,3.26.0.2
H2O cluster version age:,1 month and 6 days
H2O cluster name:,H2O_from_python_ec2_user_pu4l3l
H2O cluster total nodes:,1
H2O cluster free memory:,902 Mb
H2O cluster total cores:,2
H2O cluster allowed cores:,2
H2O cluster status:,"locked, healthy"


# Read Dataset

In [4]:
from sklearn.model_selection import train_test_split

data_train_org = pd.read_csv(path_datasets + "train.csv")
data_test = pd.read_csv(path_datasets + "test.csv")

data_train, data_valid = train_test_split(data_train_org, test_size=0.1, random_state=42)



# Encoding

In [5]:

# keywords  = ['soil']
# all_columns = data_train_org.columns
# col_keywords = []

# for k in keywords:
#     for col in all_columns:
#         if k.lower() in col.lower(): 
#             col_keywords.append(col)
            
            
# sequences = list(np.arange(1, 1+len(col_keywords)))
# soil_type_dict = dict(zip(['Soil_Type{}'.format(n) for n in sequences],
#                         sequences))

# data_train['Soil_Types'] = data_train[col_keywords].idxmax(axis=1)
# data_valid['Soil_Types'] = data_valid[col_keywords].idxmax(axis=1)
# data_test['Soil_Types'] = data_test[col_keywords].idxmax(axis=1)

# data_train['Soil_Types'] = data_train['Soil_Types'].map(soil_type_dict)
# data_valid['Soil_Types'] = data_valid['Soil_Types'].map(soil_type_dict)
# data_test['Soil_Types'] = data_test['Soil_Types'].map(soil_type_dict)

In [5]:
htrain_frame = h2o.H2OFrame(data_train)
hvalid_frame = h2o.H2OFrame(data_valid)
htest_frame = h2o.H2OFrame(data_test)
# htrain_frame = h2o.H2OFrame(data_train.drop(columns=col_keywords))
# hvalid_frame = h2o.H2OFrame(data_valid.drop(columns=col_keywords))
# htest_frame = h2o.H2OFrame(data_test.drop(columns=col_keywords))

y = 'Cover_Type'

htrain_frame[y] = htrain_frame[y].asfactor()
hvalid_frame[y] = hvalid_frame[y].asfactor()

Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%


# Start Training

# Training 1: Ordinal

In [9]:
%%time
aml = h2o.estimators.glm.H2OGeneralizedLinearEstimator( family='ordinal',
                                                           seed=42)

aml.train(y=y, training_frame=htrain_frame)

aml.model_performance()

glm Model Build progress: |███████████████████████████████████████████████| 100%
CPU times: user 23.5 ms, sys: 215 µs, total: 23.7 ms
Wall time: 461 ms


In [10]:
predictions = aml.predict(hvalid_frame.drop(y))

accuracy = accuracy_score(data_valid[y],
                         predictions['predict'].as_data_frame())

print('Accuracy: {}'.format(accuracy))

glm prediction progress: |████████████████████████████████████████████████| 100%
Accuracy: 0.15542328042328044


# Training 2: Multinomial

In [11]:
%%time
aml = h2o.estimators.glm.H2OGeneralizedLinearEstimator( family='multinomial',
                                                           seed=42)

aml.train(y=y, training_frame=htrain_frame)

aml.model_performance()

glm Model Build progress: |███████████████████████████████████████████████| 100%
CPU times: user 98.5 ms, sys: 1.2 ms, total: 99.7 ms
Wall time: 6.54 s


In [12]:
predictions = aml.predict(hvalid_frame.drop(y))

accuracy = accuracy_score(data_valid[y],
                         predictions['predict'].as_data_frame())

print('Accuracy: {}'.format(accuracy))

glm prediction progress: |████████████████████████████████████████████████| 100%
Accuracy: 0.708994708994709


# Prediction

In [ ]:
%%time
prediction_test_hframe = aml.predict(htest_frame)
# prediction_test_hframe = aml.predict(hvalid_frame)


submission = pd.DataFrame.from_dict({'ID': data_test['Id'].tolist(),
                                    'Cover_Type': prediction_test_hframe['predict'].as_data_frame().iloc[:,0].tolist(),
                                    })

submission.to_csv('./submission.csv', index=False)

In [ ]:
print(submission.head())

print()
!head submission.csv